#**1. Intaller le module d'explicabilité "tf-explain"**

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

!rm -rf sample_data

Sat Apr 29 20:58:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install pytorch-gradcam
!pip install grad-cam
!pip install --upgrade torch torchvision
!pip install omnixai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-gradcam: filename=pytorch_gradcam-0.2.1-py3-none-any.whl size=5267 sha256=c9b1d2293b227f16151ec78103ee45b274cdd7bb490a1e3194a5ac0e474b5c31
  Stored in directory: /root/.cache/pip/wheels/6f/f1/8f/96c81d13f617841f23cae192a77fea3e9e988d058ba9414f2c
Successfully built pytorch-gradcam
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.6-py3-none-any.whl size=38262 sha256=5a70f2337bd18418441a2ef5cd781ab6f1c0d2752652decfac20870073b35391
  Sto

#**2. Importer les libraries**

In [19]:
import torch
import cv2
from PIL import Image
from torchvision import transforms
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import efficientnet_b4
import numpy as np
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
from google.colab import files

#**3. Charger la base d'images de test**

In [3]:
!rm -rf sample_data
!wget --no-check-certificate https://download.smartappli.eu/test.tar
! tar xf test.tar --one-top-level
! rm test.tar

--2023-04-30 10:33:53--  https://download.smartappli.eu/test.tar
Resolving download.smartappli.eu (download.smartappli.eu)... 46.105.57.169, 2001:41d0:301::20
Connecting to download.smartappli.eu (download.smartappli.eu)|46.105.57.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58196992 (56M) [application/x-tar]
Saving to: ‘test.tar’

test.tar            100%[===================>]  55.50M  21.3MB/s    in 2.6s    

2023-04-30 10:33:56 (21.3 MB/s) - ‘test.tar’ saved [58196992/58196992]



#**4. Charger le modèle**

In [26]:
files.upload()

Saving firenet-7.pt to firenet-7.pt


In [27]:
# Load pretrained model or your own
model = torch.load("firenet-7.pt")

#**5. Expliquer le modèle avec la méthode XAI "GradCAM"**

In [30]:
rgb_img = cv2.imread('test/Test_Dataset/fire/test001.jpg')
rgb_img = cv2.resize(rgb_img, (224, 224))
rgb_img = np.float32(rgb_img) / 255
input_tensor = preprocess_image(rgb_img, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

#print(model)

#print(model.features[-1]) # dernière couche de convolution

target_layers = [model.features[-1]]
model = model.to('cpu')
cam = GradCAM(model=model, target_layers=target_layers)

#targets = [ClassifierOutputTarget(281)]
targets = [ClassifierOutputTarget(0)]  # 0 = fire, 1 = no_fire, 2 = start_fire

print(input_tensor.shape)

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
cv2.imwrite(f'GradCam.jpg', visualization)

torch.Size([1, 3, 224, 224])


True

In [33]:
cam_port = "videoTest.mp4"
cut = cv2.VideoCapture(cam_port)
out = cv2.VideoWriter('videoTest.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, (224, 224))

cpt = 0

while(cut.isOpened()):
  success, img = cut.read()
  if not success:
    break
  rgb_img = img
  rgb_img = cv2.resize(rgb_img, (224, 224))
  rgb_img = np.float32(rgb_img) / 255

  input_tensor = preprocess_image(rgb_img, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

  target_layers = [model.features[-1]]
  model = model.to('cpu')
  cam = GradCAM(model=model, target_layers=target_layers)
  
  targets = [ClassifierOutputTarget(0)]  # 0 = fire, 1 = no_fire, 2 = start_fire  

  # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
  grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

  # In this example grayscale_cam has only one image in the batch:
  grayscale_cam = grayscale_cam[0, :]
  visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
  cpt+=1
  #cv2.imwrite(f'images_xai/GradCam{cpt}.jpg', visualization) 
  out.write(visualization) 

out.release()
cut.release()
cv2.destroyAllWindows()


In [25]:
#!mkdir images_grad
#!mv *.jpg images_grad/
#!zip -r ./archive.zip ./images_grad/ 

  adding: images_grad/ (stored 0%)
  adding: images_grad/GradCam188.jpg (deflated 1%)
  adding: images_grad/GradCam170.jpg (deflated 1%)
  adding: images_grad/GradCam317.jpg (deflated 1%)
  adding: images_grad/GradCam240.jpg (deflated 1%)
  adding: images_grad/GradCam330.jpg (deflated 1%)
  adding: images_grad/GradCam279.jpg (deflated 1%)
  adding: images_grad/GradCam164.jpg (deflated 1%)
  adding: images_grad/GradCam257.jpg (deflated 1%)
  adding: images_grad/GradCam196.jpg (deflated 1%)
  adding: images_grad/GradCam175.jpg (deflated 1%)
  adding: images_grad/GradCam281.jpg (deflated 1%)
  adding: images_grad/GradCam284.jpg (deflated 1%)
  adding: images_grad/GradCam36.jpg (deflated 2%)
  adding: images_grad/GradCam104.jpg (deflated 1%)
  adding: images_grad/GradCam312.jpg (deflated 1%)
  adding: images_grad/GradCam52.jpg (deflated 2%)
  adding: images_grad/GradCam216.jpg (deflated 1%)
  adding: images_grad/GradCam147.jpg (deflated 1%)
  adding: images_grad/GradCam135.jpg (deflated 1%

In [32]:
!zip -r ./archive_xai.zip ./images_xai/

  adding: images_xai/ (stored 0%)
  adding: images_xai/GradCam188.jpg (deflated 1%)
  adding: images_xai/GradCam170.jpg (deflated 1%)
  adding: images_xai/GradCam317.jpg (deflated 1%)
  adding: images_xai/GradCam240.jpg (deflated 1%)
  adding: images_xai/GradCam330.jpg (deflated 1%)
  adding: images_xai/GradCam279.jpg (deflated 1%)
  adding: images_xai/GradCam164.jpg (deflated 1%)
  adding: images_xai/GradCam257.jpg (deflated 1%)
  adding: images_xai/GradCam196.jpg (deflated 1%)
  adding: images_xai/GradCam175.jpg (deflated 1%)
  adding: images_xai/GradCam281.jpg (deflated 1%)
  adding: images_xai/GradCam284.jpg (deflated 1%)
  adding: images_xai/GradCam36.jpg (deflated 2%)
  adding: images_xai/GradCam104.jpg (deflated 1%)
  adding: images_xai/GradCam312.jpg (deflated 1%)
  adding: images_xai/GradCam52.jpg (deflated 2%)
  adding: images_xai/GradCam216.jpg (deflated 1%)
  adding: images_xai/GradCam147.jpg (deflated 2%)
  adding: images_xai/GradCam135.jpg (deflated 2%)
  adding: images_x